merge all metadata sheets and generate universal metadata file.

In [2]:
import glob
import pandas as pd
import av
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import calendar
from tqdm import tqdm
import utils

# dataset_dir = "/home/arthur/felles/Research/Users/Alexander/Still Standing"
dataset_dir = "/fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing"

In [3]:
metadata = pd.read_csv(os.path.join(dataset_dir, "metadata/stillstanding_output.csv"))[["StillStandingNo", "Date", "Audio Clap Start", "Audio Clap End", "Mobile Timecode", "Mobile Clap Start", "Mobile Clap End", "Video Clap Start", "Video Clap End", "Video Crop Height", "Video Crop Width", "Video Crop X Position", "Video Crop Y Position", "Video Crop Ratio", "Watch Timecode"]]
metadata

,StillStandingNo,Date,Audio Clap Start,Audio Clap End,Mobile Timecode,Mobile Clap Start,Mobile Clap End,Video Clap Start,Video Clap End,Video Crop Height,Video Crop Width,Video Crop X Position,Video Crop Y Position,Video Crop Ratio,Watch Timecode
0,1,2023-01-01,0:01:47,0:11:18,2023-01-01 11:02:02,22.0,598.0,00:01:33,00:11:04,1530.0,637.0,380.0,1100.0,2.40188383045526,2023-01-01 11:02:13
1,2,2023-01-02,0:00:57,0:10:24,2023-01-02 11:59:09,26.0,593.0,00:01:02,00:10:28,1530.0,637.0,305.0,990.0,2.40188383045526,2023-01-02 11:59:32
2,3,2023-01-03,0:01:19,0:10:46,2023-01-03 11:35:29,24.0,593.0,00:01:10,00:10:38,1530.0,637.0,370.0,980.0,2.40188383045526,2023-01-03 11:35:48
3,4,2023-01-04,0:02:11,0:11:43,2023-01-04 12:34:10,21.0,592.0,00:02:07,00:11:39,1210.0,504.0,410.0,1130.0,2.40079365079365,2023-01-04 12:34:25
4,5,2023-01-05,0:01:05,0:10:39,2023-01-05 11:43:41,22.0,593.0,00:01:11,00:10:47,1330.0,554.0,460.0,1060.0,2.40072202166065,2023-01-05 11:43:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,361,2023-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#DIV/0!,NaN
361,362,2023-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#DIV/0!,NaN
362,363,2023-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#DIV/0!,NaN
363,364,2023-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#DIV/0!,NaN


In [4]:
# Add source file path
source_audio_path = []
source_video_LRV_1_path = []
source_video_LRV_2_path = []
source_video_LRV_3_path = []
source_video_LRV_4_path = []
# source_video_THM_1_path = []
# source_video_THM_2_path = []
# source_video_THM_3_path = []
source_video_360_1_path = []
source_video_360_2_path = []
source_video_360_3_path = []
source_video_360_4_path = []
source_watch_csv_path = []
source_watch_tcx_path = []
source_phone_path = []

for idx, row in tqdm(metadata.iterrows()):
    _, month, day = row["Date"].split("-")
    date_path = os.path.join(dataset_dir, "1-raw", month, f"2023-{month}-{day}", "1-raw")

    # audio
    audio = glob.glob(f"{date_path}/*.WAV")
    if len(audio) == 1:
        source_audio_path.append(audio[0].split("Still Standing/")[1])
    else:
        print(f"Audio file error: {date_path}")
        source_audio_path.append(None)

    # video_lrv
    video_LRV = glob.glob(f"{date_path}/*.LRV")
    if len(video_LRV) == 2:
        video_LRV = sorted(video_LRV)
        source_video_LRV_1_path.append(video_LRV[0].split("Still Standing/")[1])
        source_video_LRV_2_path.append(video_LRV[1].split("Still Standing/")[1])
        source_video_LRV_3_path.append(None)
        source_video_LRV_4_path.append(None)
    elif len(video_LRV) == 3:
        video_LRV = sorted(video_LRV)
        source_video_LRV_1_path.append(video_LRV[0].split("Still Standing/")[1])
        source_video_LRV_2_path.append(video_LRV[1].split("Still Standing/")[1])
        source_video_LRV_3_path.append(video_LRV[2].split("Still Standing/")[1])
        source_video_LRV_4_path.append(None)
    elif len(video_LRV) == 4:
        video_LRV = sorted(video_LRV)
        source_video_LRV_1_path.append(video_LRV[0].split("Still Standing/")[1])
        source_video_LRV_2_path.append(video_LRV[1].split("Still Standing/")[1])
        source_video_LRV_3_path.append(video_LRV[2].split("Still Standing/")[1])
        source_video_LRV_4_path.append(video_LRV[3].split("Still Standing/")[1])
    else:
        print(f"Video LRV file error: {date_path}")
        source_video_LRV_1_path.append(None)
        source_video_LRV_2_path.append(None)
        source_video_LRV_3_path.append(None)
        source_video_LRV_4_path.append(None)

    # # video_thm
    # video_THM = glob.glob(f"{date_path}/*.THM")
    # if len(video_THM) == 2:
    #     video_THM = sorted(video_THM)
    #     source_video_THM_1_path.append(video_THM[0].split("Still Standing/")[1])
    #     source_video_THM_2_path.append(video_THM[1].split("Still Standing/")[1])
    #     source_video_LRV_3_path.append(None)
    # elif len(video_THM) == 3:
    #     video_THM = sorted(video_THM)
    #     source_video_THM_1_path.append(video_THM[0].split("Still Standing/")[1])
    #     source_video_THM_2_path.append(video_THM[1].split("Still Standing/")[1])
    #     source_video_THM_3_path.append(video_THM[2].split("Still Standing/")[1])
    # else:
    #     print(f"Video THM file error: {date_path}")
    #     source_video_THM_1_path.append(None)
    #     source_video_THM_2_path.append(None)

    # video_360
    video_360 = glob.glob(f"{date_path}/*.360")
    if len(video_360) == 2:
        video_360 = sorted(video_360)
        source_video_360_1_path.append(video_360[0].split("Still Standing/")[1])
        source_video_360_2_path.append(video_360[1].split("Still Standing/")[1])
        source_video_360_3_path.append(None)
        source_video_360_4_path.append(None)
    elif len(video_360) == 3:
        video_360 = sorted(video_360)
        source_video_360_1_path.append(video_360[0].split("Still Standing/")[1])
        source_video_360_2_path.append(video_360[1].split("Still Standing/")[1])
        source_video_360_3_path.append(video_360[2].split("Still Standing/")[1])
        source_video_360_4_path.append(None)
    elif len(video_360) == 4:
        video_360 = sorted(video_360)
        source_video_360_1_path.append(video_360[0].split("Still Standing/")[1])
        source_video_360_2_path.append(video_360[1].split("Still Standing/")[1])
        source_video_360_3_path.append(video_360[2].split("Still Standing/")[1])
        source_video_360_4_path.append(video_360[3].split("Still Standing/")[1])
    else:
        print(f"Video 360 file error: {date_path}")
        source_video_360_1_path.append(None)
        source_video_360_2_path.append(None)
        source_video_360_3_path.append(None)
        source_video_360_4_path.append(None)

    # watch_csv
    watch_csv = glob.glob(f"{date_path}/*polar.CSV")
    if len(watch_csv) == 1:
        source_watch_csv_path.append(watch_csv[0].split("Still Standing/")[1])
    else:
        print(f"Watch CSV file error: {date_path}")
        source_watch_csv_path.append(None)

    # watch_tcx
    watch_tcx = glob.glob(f"{date_path}/*polar.TCX")
    if len(watch_tcx) == 1:
        source_watch_tcx_path.append(watch_tcx[0].split("Still Standing/")[1])
    else:
        print(f"Watch TCX file error: {date_path}")
        source_watch_tcx_path.append(None)

    # phone
    phone = glob.glob(f"{date_path}/*T*.csv")
    if len(phone) == 1:
        source_phone_path.append(phone[0].split("Still Standing/")[1])
    else:
        print(f"Phone file error: {date_path}")
        source_phone_path.append(None)

    # processed_video_spherical
    processed_video_spherical = glob.glob(f"{date_path}/*spherical.mp4")


metadata["source_audio_path"] = source_audio_path
metadata["source_video_LRV_1_path"] = source_video_LRV_1_path
metadata["source_video_LRV_2_path"] = source_video_LRV_2_path
metadata["source_video_LRV_3_path"] = source_video_LRV_3_path
metadata["source_video_LRV_4_path"] = source_video_LRV_4_path
metadata["source_video_360_1_path"] = source_video_360_1_path
metadata["source_video_360_2_path"] = source_video_360_2_path
metadata["source_video_360_3_path"] = source_video_360_3_path
metadata["source_video_360_4_path"] = source_video_360_4_path
metadata["source_watch_csv_path"] = source_watch_csv_path
metadata["source_phone_path"] = source_phone_path

/tmp/ipykernel_2955074/2008063632.py:18: TqdmMonitorWarning: tqdm:disabling monitor support (monitor_interval = 0) due to:
can't start new thread
  for idx, row in tqdm(metadata.iterrows()):
67it [00:00, 112.12it/s]

Audio file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/02/2023-02-14/1-raw
Video LRV file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/02/2023-02-14/1-raw
Video 360 file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/02/2023-02-14/1-raw
Watch CSV file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/02/2023-02-14/1-raw
Watch TCX file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/02/2023-02-14/1-raw
Phone file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/02/2023-02-14/1-raw


138it [00:01, 112.23it/s]

Audio file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/05/2023-05-06/1-raw
Video LRV file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/05/2023-05-06/1-raw
Video 360 file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/05/2023-05-06/1-raw
Phone file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/05/2023-05-06/1-raw


197it [00:01, 108.68it/s]

Audio file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/07/2023-07-01/1-raw
Video LRV file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/07/2023-07-01/1-raw
Audio file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/07/2023-07-02/1-raw
Video LRV file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/07/2023-07-02/1-raw
Audio file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/07/2023-07-03/1-raw
Video LRV file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/07/2023-07-03/1-raw
Audio file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/07/2023-07-04/1-raw
Video LRV file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/07/2023-07-04/1-raw
Audio file error: /fp/homes01/u01/ec-jin

244it [00:02, 104.90it/s]

Audio file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/08/2023-08-12/1-raw


365it [00:03, 109.10it/s]

Audio file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/11/2023-11-24/1-raw
Video LRV file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/12/2023-12-15/1-raw
Audio file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/12/2023-12-29/1-raw


In [5]:
# processed
processed_video_spherical_path = []
processed_audio_path = []
processed_watch_path = []
processed_phone_path = []

for idx, row in tqdm(metadata.iterrows()):
    _, month, day = row["Date"].split("-")
    date_path = os.path.join(dataset_dir, "1-raw", month, f"2023-{month}-{day}", "2-processed")

    # processed_video_spherical
    processed_video_spherical = glob.glob(f"{date_path}/*spherical.mp4")
    if len(processed_video_spherical) == 1:
        processed_video_spherical_path.append(processed_video_spherical[0].split("Still Standing/")[1])
    else:
        print(f"Processed video spherical file error: {date_path}")
        processed_video_spherical_path.append(None)

    # processed_audio
    processed_audio = glob.glob(f"{date_path}/*ambisonics.wav")
    if len (processed_audio) == 1:
        processed_audio_path.append(processed_audio[0].split("Still Standing/")[1])
    else:
        print(f"Processed audio file error: {date_path}")
        processed_audio_path.append(None)

    # processed_watch
    processed_watch = glob.glob(f"{date_path}/*watch.csv") + glob.glob(f"{date_path}/*polar.csv")
    if len(processed_watch) == 1:
        processed_watch_path.append(processed_watch[0].split("Still Standing/")[1])
    else:
        print(f"Processed watch file error: {date_path}")
        processed_watch_path.append(None)

    # processed_phone
    processed_phone = glob.glob(f"{date_path}/*phone.csv")
    if len(processed_phone) == 1:
        processed_phone_path.append(processed_phone[0].split("Still Standing/")[1])
    else:
        print(f"Processed phone file error: {date_path}")
        processed_phone_path.append(None)
    

metadata["processed_video_spherical_path"] = processed_video_spherical_path
metadata["processed_audio_path"] = processed_audio_path
metadata["processed_watch_path"] = processed_watch_path
metadata["processed_phone_path"] = processed_phone_path

35it [00:00, 84.79it/s]

Processed audio file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/01/2023-01-20/2-processed
Processed audio file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/01/2023-01-29/2-processed


53it [00:00, 83.26it/s]

Processed video spherical file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/02/2023-02-14/2-processed
Processed audio file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/02/2023-02-14/2-processed
Processed watch file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/02/2023-02-14/2-processed
Processed phone file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/02/2023-02-14/2-processed
Processed audio file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/02/2023-02-16/2-processed


106it [00:01, 92.22it/s]

...

233it [00:03, 64.45it/s]

Processed audio file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/08/2023-08-12/2-processed


336it [00:04, 61.80it/s]

Processed audio file error: /fp/homes01/u01/ec-jinyueg/felles/Research/Users/Alexander/Still Standing/1-raw/11/2023-11-24/2-processed


365it [00:05, 69.15it/s]


In [6]:
metadata[metadata.processed_video_spherical_path.isnull()][['StillStandingNo', 'Date']]

,StillStandingNo,Date
44,45,2023-02-14
181,182,2023-07-01
182,183,2023-07-02
183,184,2023-07-03
184,185,2023-07-04
185,186,2023-07-05
186,187,2023-07-06
187,188,2023-07-07
188,189,2023-07-08
189,190,2023-07-09


In [6]:
# get video duration
source_video_LRV_1_duration = []
source_video_LRV_2_duration = []
source_video_LRV_3_duration = []
source_video_LRV_4_duration = []
source_video_360_1_duration = []
source_video_360_2_duration = []
source_video_360_3_duration = []
source_video_360_4_duration = []
processed_video_spherical_duration = []

for idx, row in tqdm(metadata.iterrows()):
    if row["source_video_LRV_1_path"]:
        source_video_LRV_1_duration.append(utils.get_video_duration(os.path.join(dataset_dir, row["source_video_LRV_1_path"])))
    else:
        source_video_LRV_1_duration.append(None)
    if row["source_video_LRV_2_path"]:
        source_video_LRV_2_duration.append(utils.get_video_duration(os.path.join(dataset_dir, row["source_video_LRV_2_path"])))
    else:
        source_video_LRV_2_duration.append(None)
    if row["source_video_LRV_3_path"]:
        source_video_LRV_3_duration.append(utils.get_video_duration(os.path.join(dataset_dir, row["source_video_LRV_3_path"])))
    else:
        source_video_LRV_3_duration.append(None)
    if row["source_video_LRV_4_path"]:
        source_video_LRV_4_duration.append(utils.get_video_duration(os.path.join(dataset_dir, row["source_video_LRV_4_path"])))
    else:
        source_video_LRV_4_duration.append(None)
    if row["source_video_360_1_path"]:
        source_video_360_1_duration.append(utils.get_video_duration(os.path.join(dataset_dir, row["source_video_360_1_path"])))
    else:
        source_video_360_1_duration.append(None)
    if row["source_video_360_2_path"]:
        source_video_360_2_duration.append(utils.get_video_duration(os.path.join(dataset_dir, row["source_video_360_2_path"])))
    else:
        source_video_360_2_duration.append(None)
    if row["source_video_360_3_path"]:
        source_video_360_3_duration.append(utils.get_video_duration(os.path.join(dataset_dir, row["source_video_360_3_path"])))
    else:
        source_video_360_3_duration.append(None)
    if row["source_video_360_4_path"]:
        source_video_360_4_duration.append(utils.get_video_duration(os.path.join(dataset_dir, row["source_video_360_4_path"])))
    else:
        source_video_360_4_duration.append(None)
    if row["processed_video_spherical_path"]:
        processed_video_spherical_duration.append(utils.get_video_duration(os.path.join(dataset_dir, row["processed_video_spherical_path"])))
    else:
        processed_video_spherical_duration.append(None)
        
metadata["source_video_LRV_1_duration"] = source_video_LRV_1_duration
metadata["source_video_LRV_2_duration"] = source_video_LRV_2_duration
metadata["source_video_LRV_3_duration"] = source_video_LRV_3_duration
metadata["source_video_LRV_4_duration"] = source_video_LRV_4_duration
metadata["source_video_360_1_duration"] = source_video_360_1_duration
metadata["source_video_360_2_duration"] = source_video_360_2_duration
metadata["source_video_360_3_duration"] = source_video_360_3_duration
metadata["source_video_360_4_duration"] = source_video_360_4_duration
metadata["processed_video_spherical_duration"] = processed_video_spherical_duration

221it [05:05,  1.35s/it]

Error: [Errno 1094995529] Invalid data found when processing input: '/home/arthur/felles/Research/Users/Alexander/Still Standing/1-raw/08/2023-08-10/1-raw/2023-08-10-GS010371.360'
Error: [Errno 1094995529] Invalid data found when processing input: '/home/arthur/felles/Research/Users/Alexander/Still Standing/1-raw/08/2023-08-10/1-raw/2023-08-10-GS020371.360'


273it [06:18,  1.39s/it]

Error: [Errno 1094995529] Invalid data found when processing input: '/home/arthur/felles/Research/Users/Alexander/Still Standing/1-raw/10/2023-10-01/1-raw/2023-10-01-GS010426.LRV'
Error: [Errno 1094995529] Invalid data found when processing input: '/home/arthur/felles/Research/Users/Alexander/Still Standing/1-raw/10/2023-10-01/1-raw/2023-10-01-GS020426.360'


365it [08:19,  1.37s/it]


In [7]:
# find video with different duration between LRV and spherical
lrv_duration = metadata.source_video_LRV_1_duration + metadata.source_video_LRV_2_duration + metadata.source_video_LRV_3_duration.fillna(0) + metadata.source_video_LRV_4_duration.fillna(0)
metadata["source_video_LRV_duration"] = lrv_duration
diff = metadata.source_video_LRV_duration - metadata.processed_video_spherical_duration
metadata[diff.abs() > 0.01][["StillStandingNo", "Date", "source_video_LRV_duration", "processed_video_spherical_duration"]]

,StillStandingNo,Date,source_video_LRV_duration,processed_video_spherical_duration
0,1,2023-01-01,707.106400,707.120400
1,2,2023-01-02,665.798467,665.812467
2,3,2023-01-03,651.684367,651.698367
194,195,2023-07-14,820.400000,820.410667
258,259,2023-09-16,1010.080000,1010.090667
260,261,2023-09-18,717.080000,717.106667
331,332,2023-11-28,1090.280000,1090.600000


In [7]:
metadata.to_csv("../metadata.csv",index=False)